# Hydrogen Simulation Replication

## Authentication with Quantum Inspire

In [1]:
import logging
import os

from projectq import MainEngine
from projectq.setups import linear
from projectq.ops import H, Rx, Rz, CNOT, CZ, Measure, All

from qiskit.visualization import plot_histogram

from quantuminspire.credentials import get_token_authentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.projectq.backend_qx import QIBackend

import getpass

# import builder classes
from physicalBuilder import get_builder_type_by_name

QI_URL = os.getenv('API_URL', 'https://api.quantum-inspire.com')

In [ ]:
API_TOKEN = getpass.getpass("Please enter a valid API Token: ")
auth = get_token_authentication(API_TOKEN)
qi_api = QuantumInspireAPI(QI_URL, auth)

projectq_backend = QIBackend(quantum_inspire_api=qi_api)

## 1-Qubit Replicated Circuit

In [ ]:


qasm = """version 1.0
qubits 4
prep_z q[0]
prep_z q[1]
prep_z q[2]
prep_z q[3]

CNOT q[0], q[1]
CNOT q[1], q[2]
CNOT q[2], q[3]

Rz q[0], -1.12
Rz q[1], -1.12
Rz q[2], -1.12
Rz q[3], -1.12

CNOT q[2], q[3]
CNOT q[1], q[2]
CNOT q[0], q[1]

Rz q[0], -1.12
Rz q[1], -1.12
Rz q[2], -1.12

CNOT q[0], q[1]
Rz q[1], -1.12
CNOT q[0], q[1]

# end of the first section
# second section

CNOT q[0], q[2]
Rz q[2], -1.12
CNOT q[0], q[2]

CNOT q[1], q[3]
Rz q[3], -1.12
CNOT q[1], q[3]

H q[0]
H q[2]

CNOT q[0], q[1]
CNOT q[1], q[2]
Rz q[2], -1.12
CNOT q[1], q[2]
CNOT q[0], q[1]

Rx q[0],1.12
Rx q[2],1.12

CNOT q[0], q[1]
CNOT q[1], q[2]
Rz q[2], -1.12
CNOT q[1], q[2]
CNOT q[0], q[1]

# end of the second section
# third section

CNOT q[0], q[2]
CNOT q[1], q[3]
Rz q[3], -1.12
CNOT q[1], q[3]
CNOT q[0], q[2]

CNOT q[1], q[2]
CNOT q[2], q[3]
Rz q[3], -1.12
CNOT q[2], q[3]
CNOT q[1], q[2]

H q[0]
H q[2]

CNOT q[0], q[1]
CNOT q[1], q[2]
CNOT q[2], q[3]
Rz q[3], -1.12
CNOT q[2], q[3]
CNOT q[1], q[2]
CNOT q[0], q[1]

H q[0]
H q[2]
Rx q[0],-1.12
Rx q[2],-1.12

CNOT q[0], q[1]
CNOT q[1], q[2]
CNOT q[2], q[3]
Rz q[3], -1.12
CNOT q[2], q[3]
CNOT q[1], q[2]
CNOT q[0], q[1]

Rx q[0],1.12
Rx q[2],1.12

# end of the third section
# fourth section

CNOT q[0], q[1]
CNOT q[1], q[2]
CNOT q[2], q[3]
Rz q[3], -1.12
CNOT q[2], q[3]
CNOT q[1], q[2]
CNOT q[0], q[1]

H q[0]
H q[1]
H q[2]
H q[3]

measure_z q[0]
measure_z q[1]
measure_z q[2]
measure_z q[3]"""

backend_type = qi_api.get_backend_type_by_name('QX Single-node Simulator')
result = qi_api.execute_qasm(qasm, backend_type=backend_type, number_of_shots = 10)

if result.get('histogram', {}):
    print(result['histogram'])


In [ ]:
import quantuminspire.qiskit.tools as plot_histogram

plot_histogram(result['histogram'])


## Builder with Single-Qubit Logical Qubits

In [3]:
# Circuit Replication Using Builder

## Initialization
builder = get_builder_type_by_name("2 qubit")(qubit_num = 4, version="1.0")

# First Section
builder.addCNOT([0,1,2], [1,2,3], True)
builder.embed()
builder.addRotation(["z"] * 4, [0,1,2,3], [-1.12] * 4)
builder.pop(n = 3)

builder.addRotation(["z"] * 3, [0,1,2], [-1.12] * 3)

builder.addCNOT(0, 1, True)
builder.embed()
builder.addRotation("z", 1, -1.12)
builder.pop()

# Second Section
builder.addCNOT(0, 2, True)
builder.embed()
builder.addRotation("z", 2, -1.12)
builder.pop()

builder.addCNOT(1, 3, True)
builder.embed()
builder.addRotation("z", 3, -1.12)
builder.pop

builder.addH([0, 2])

builder.addCNOT([0, 1], [1, 2], True)
builder.embed()
builder.addRotation("z", 2, -1.12)
builder.pop(n = 2)

# Third Section
builder.addCNOT([0, 1], [2, 3], True)
builder.embed()
builder.addRotation("z", 3, -1.12)
builder.pop(n = 2)

builder.addCNOT([1, 2], [2, 3], True)
builder.embed()
builder.addRotation("z", 3, -1.12)
builder.pop(n = 2)

builder.addH([0, 2])

builder.addCNOT([0,1,2], [1,2,3], True)
builder.embed()
builder.addRotation("z", 3, -1.12)
builder.pop(n = 3)

builder.addH([0, 2])
builder.addRotation(["x"] * 2, [0, 2], [-1.12] * 2)

builder.addCNOT([0,1,2], [1,2,3], True)
builder.embed()
builder.addRotation("z", 3, -1.12)
builder.pop(n = 3)

builder.addH([0, 2])
builder.addRotation(["x"] * 2, [0, 2], [-1.12] * 2)

# Fourth Section
builder.addCNOT([0,1,2], [1,2,3], True)
builder.embed()
builder.addRotation("z", 3, -1.12)
builder.pop(n = 3)

builder.addH(list(range(4)))

qasm = builder.build()

In [ ]:
backend_type = qi_api.get_backend_type_by_name('QX Single-node Simulator')
result = qi_api.execute_qasm(qasm, backend_type=backend_type, number_of_shots = 10)

if result.get('histogram', {}):
    print(result['histogram'])